In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from torchvision import datasets, transforms
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import glob
import random
import cv2
import imutils
from skimage import color
from skimage.util.dtype import convert
import import_ipynb #pip install import-ipynb

from data import ImageNet, VisionDataset
from CUnet import CUNet
from eccv16 import eccv16

In [17]:
cu_model = CUNet()
ecc_model = eccv16()

In [18]:
trans = transforms.Compose([transforms.ToTensor()])
ImageNet_train_dataset = ImageNet(root="./imagenet/train", transform=trans)
ImageNet_eval_dataset = ImageNet(root="./imagenet/val", transform=trans)

In [19]:
train_dataloader_imagenet = DataLoader(ImageNet_train_dataset, batch_size=32, shuffle=True)
eval_dataloader_imagenet = DataLoader(ImageNet_eval_dataset, batch_size=32, shuffle=True)

In [4]:
EPOCHS = 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.SGD(cu_model.parameters(), lr = 0.1) #change as needed
loss_function = torch.nn.MSELoss() #change as needed

In [23]:
def train_model(dataloader, model, device, optimizer, loss_function, epochs=1):
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for batch_index,(inputs, expected) in enumerate(dataloader):
            optimizer.zero_grad() 
            inputs = inputs.to(device)
            expected = expected.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, expected)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*len(expected)

In [24]:
def eval_model(dataloader, model, device, optimizer, loss_function, epochs=1):
    model.eval()
    val_loss = 0
    for batch_index,(inputs, expected) in enumerate(dataloader):
        # inputs = torchvision.transforms.ToTensor(train_dataloader_imagenet.resize_image(lightness))
        # expected = torchvision.transforms.ToTensor(train_dataloader_imagenet.resize_image(colors))
        # inputs.type(torch.float32)
        # expected.type(torch.float32)
        print(type(inputs))
        inputs = inputs.to(device)
        expected = expected.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, expected)
        val_loss += loss.item()*len(expected)
    avg_val_loss = val_loss/len(eval_dataloader_imagenet)

In [25]:
def name_to_epoch(name):
    _, a = name.split("/")
    _, epoch, _ = name.split("_") #name, epoch, avg_l.pth
    return(int(epoch))

In [26]:
train_model(train_dataloader_imagenet, cu_model, device, optimizer, loss_function, 1)

<string>:29: FutureWarning: The use of this function is discouraged as its behavior may change dramatically in scikit-image 1.0. This function will be removed in scikit-image 1.0.


ValueError: the input array must have size 3 along `channel_axis`, got (256, 256)

In [ ]:
#filename = model/type(model)_epoch_avg_validation_loss.pth'

model = cu_model
def name_to_epoch(name):
    _, a = name.split("/")
    _, epoch, _ = name.split("_") #name, epoch, avg_l.pth
    return(int(epoch))

paths = glob.glob(f'model/{type(model).__name__}_*.pth')
#only find models that have the same type as the model being trained

paths.sort(key = name_to_epoch) #most recent epoch
start = 0
if len(paths) > 0:
    target = paths[-1]
    start = name_to_epoch(target)
    model.load_state_dict(torch.load(target)) 


# model = model.to(device)

for epoch in range(start, EPOCHS):
    model.train()
    train_loss = 0
    for batch_index,(inputs, expected) in enumerate(train_dataloader_imagenet):
        optimizer.zero_grad() 
        inputs = inputs.to(device)
        expected = expected.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, expected)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*len(expected)
    
    model.eval()
    val_loss = 0
    for batch_index,(inputs, expected) in enumerate(eval_dataloader_imagenet):
        # inputs = torchvision.transforms.ToTensor(train_dataloader_imagenet.resize_image(lightness))
        # expected = torchvision.transforms.ToTensor(train_dataloader_imagenet.resize_image(colors))
        # inputs.type(torch.float32)
        # expected.type(torch.float32)
        print(type(inputs))
        inputs = inputs.to(device)
        expected = expected.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, expected)
        val_loss += loss.item()*len(expected)
    avg_val_loss = val_loss/len(eval_dataloader_imagenet)
    filename = f'model/{type(model).__name__}_{epoch}_{avg_val_loss}.pth'
    torch.save(model.state_dict(), filename)